# Task 3 — A/B Hypothesis Testing
This notebook performs A/B hypothesis testing on insurance claim behavior.
We use statistical tests to compare:
- Gender: Male vs Female
- Provinces (multi-group test)
- High vs Low Premium
- Alarm/Tracking device vs None


In [1]:
import sys
sys.path.append("..")  # allows importing from tests folder

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from tests.test_helpers import (
    proportion_test,
    two_sample_t_or_mannwhitney,
    anova_or_kruskal
)


# Insurance Data Feature Engineering

## Overview
Load the `MachineLearningRating_v3.csv` dataset and create derived features to prepare it for analysis.

## Derived Features
- **has_claim**: Binary indicator for whether a claim was made.  
- **Margin**: Profit margin for each record (`TotalPremium - TotalClaims`).  
- **LossRatio**: Ratio of claims to premiums, with `NaN`


In [2]:
df = pd.read_csv('../data/processed/MachineLearningRating_v3.csv')

# Derived columns
df['has_claim'] = (df['TotalClaims'] > 0).astype(int)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']
df['LossRatio'] = np.where(
    df['TotalPremium'] > 0,
    df['TotalClaims'] / df['TotalPremium'],
    np.nan
)

df.head()


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,has_claim,Margin,LossRatio
0,145249,12827,2015-03-01 00:00:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0,0.000000,NaN
3,145255,12827,2015-05-01 00:00:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,NaN,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,0,0.000000,NaN


# Data Summary

## Overview
Generate descriptive statistics for key insurance features to understand distributions and summary metrics.

## Columns Summarized
- **TotalClaims** – Total claims made by each customer.  
- **TotalPremium** – Total premium paid by each customer.  
- **has_claim** – Binary indicator of whether a claim was made.  
- **Margin** – Profit margin per record (`TotalPremium - TotalClaims`).  
- **LossRatio** – Ratio of claims to premiums.


In [3]:
df[['TotalClaims','TotalPremium','has_claim','Margin','LossRatio']].describe()


,TotalClaims,TotalPremium,has_claim,Margin,LossRatio
count,256960.000000,256960.000000,256960.000000,256960.000000,169089.000000
mean,70.356984,65.241273,0.003067,-5.115711,0.359912
std,2393.816095,161.154763,0.055292,2384.365620,7.984171
min,-635.480000,-80.409357,0.000000,-303520.451754,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2.476667,0.000000,2.444474,0.000000
75%,0.000000,25.375940,0.000000,21.929825,0.000000
max,304338.657895,1561.080439,1.000000,1561.080439,1485.751642


# Gender Distribution

## Overview
Count the occurrences of each value in the `Gender` column, including missing values (`NaN`), to understand the distribution of customer genders.


In [9]:
df['Gender'].value_counts(dropna=False)


Gender
Not specified    253463
Male               3497
Name: count, dtype: int64

# Gender-Based Claim Analysis

## Overview
Compare insurance claim behavior between male customers and those with unspecified gender.

## Steps
1. **Split Data**
   - `male` → customers with `Gender` = 'Male'  
   - `not_spec` → customers with `Gender` = 'Not specified'

2. **Proportion Test**
```python
stat, pval = proportion_test(male['has_claim'], not_spec['has_claim'])


In [10]:
male = df[df['Gender'] == 'Male']
not_spec = df[df['Gender'] == 'Not specified']

# Proportion test
stat, pval = proportion_test(male['has_claim'], not_spec['has_claim'])
print("Proportion Z-test:", stat, pval)

# LossRatio test
male_loss = male['LossRatio'].dropna()
not_spec_loss = not_spec['LossRatio'].dropna()

test, stat, pval = two_sample_t_or_mannwhitney(male_loss, not_spec_loss)
print(test, stat, pval)


Proportion Z-test: 3.472320212974917 0.0005159804874381079
Mann-Whitney U 243118538.0 0.012283433352089115


# Premium-Based Claim Analysis

## Overview
Compare insurance claim behavior between customers with high premiums and those with low premiums.

## Steps
1. **Split Data by Premium**
   - `median_premium` → median of `TotalPremium`  
   - `high` → customers with `TotalPremium` above median  
   - `low` → customers with `TotalPremium` at or below median

2. **Proportion Test**
```python
proportion_test(high['has_claim'], low['has_claim'])


In [11]:
median_premium = df['TotalPremium'].median()
high = df[df['TotalPremium'] > median_premium]
low = df[df['TotalPremium'] <= median_premium]

proportion_test(high['has_claim'], low['has_claim'])
two_sample_t_or_mannwhitney(
    high['LossRatio'].dropna(),
    low['LossRatio'].dropna()
)


('Mann-Whitney U', np.float64(2624491452.0), np.float64(3.201208335139909e-54))

# Alarm-Based Claim Analysis

## Overview
Compare insurance claim behavior between customers with and without an alarm, performing statistical tests only if sample sizes and claim counts are sufficient.

## Steps
1. **Check Group Sizes**
```python
if len(with_alarm) > 5 and len(without_alarm) > 5:


In [ ]:
if len(with_alarm) > 5 and len(without_alarm) > 5:
    if with_alarm['has_claim'].sum() > 0 or without_alarm['has_claim'].sum() > 0:
        stat, pval = proportion_test(with_alarm['has_claim'], without_alarm['has_claim'])
        print("Proportion test:", stat, pval)

        test, stat, pval = two_sample_t_or_mannwhitney(
            with_alarm['LossRatio'].dropna(),
            without_alarm['LossRatio'].dropna()
        )
        print(test, stat, pval)
    else:
        print("No claims in either group — cannot perform proportion test")
else:
    print("One or both groups too small — skipping test")


One or both groups too small — skipping test


# Province-Based Loss Ratio Analysis

## Overview
Compare `LossRatio` across different provinces to test if distributions differ significantly.

## Steps
1. **Group Data by Province**
```python
groups = [g['LossRatio'].dropna() for _, g in df.groupby('Province')]


In [14]:
groups = [g['LossRatio'].dropna() for _, g in df.groupby('Province')]
anova_or_kruskal(groups)


('ANOVA', np.float64(3.5349147749817873), np.float64(0.00042426831075107765))

# Task 3 — A/B Hypothesis Testing Summary

This notebook performed statistical tests on insurance claim behavior. Key results:



### 1. Male vs Not Specified
- **Proportion Z-test:** `3.4723, p=0.0005` → statistically significant difference.
- **Loss Ratio (Mann-Whitney U):** `243,118,538.0, p=0.0123` → statistically significant difference.
- ⚠️ Only males and “Not Specified” groups considered; females missing or very small.

---

### 2. Alarm / Immobiliser
- **Result:** One or both groups too small — skipping test.
- ⚠️ Cannot conclude; not enough data for either group.

---

### 3. High vs Low Premium
- **Loss Ratio (Mann-Whitney U):** `2,624,491,452.0, p≈3.20e-54` → highly significant difference.
- ✅ High premium customers have higher loss ratios.

---

### 4. Provinces (Multi-group test)
- **ANOVA/Kruskal:** `ANOVA, F=3.5349, p=0.0004` → significant difference in loss ratios across provinces.
- ✅ Provincial risk varies statistically.

---

### ✅ **Conclusion**
1. High premium clients tend to have higher losses.
2. Some categorical groups are too small to test (Alarm / Immobiliser).
3. Provincial differences exist — geography matters.
4. Gender differences significant between Male and Not Specified.

> **Next steps for Task 4:** Feature engineering, handling missing values, encoding, creating derived features, and preparing dataset for modeling.
